In [ ]:
import requests
import time  

#OAuth Access Token
auth = ""

#DB Path
output_file_path=""

# rate limit interval in seconds
RATE_LIMIT_INTERVAL = 1 
    

#DB Path
output_file_path=""

# rate limit interval in seconds
RATE_LIMIT_INTERVAL = 1  

# Rate Limiting 
def slack_api_request(url, method, headers=None, params=None, data=None):
    
    # delay to respect the rate limit
    time.sleep(RATE_LIMIT_INTERVAL)
    
    # Make the API request
    response = requests.request(method, url, headers=headers, params=params, data=data)
    
    return response

# This returns a list of all channels
def channel_list(auth):
    url = "https://slack.com/api/conversations.list"
    headers = {
        "Authorization": f"Bearer {auth}"
    }

    #Uses the API call funtion to respect rate limits
    response = slack_api_request(url, "GET", headers=headers)
    data = response.json()

    if response.status_code == 200 and data["ok"]:
        return data["channels"]
    else:
        print("Error fetching channels:", data.get("error", "Unknown error"))
        return []

    
# Check if bot is in the channel, and if not, invite the bot
def ensure_bot_in_channel(channel_id, auth):
    url = "https://slack.com/api/conversations.members"
    headers = {
        "Authorization": f"Bearer {auth}"
    }
    params = {
        "channel": channel_id
    }
     
    #Uses the API call funtion to respect rate limits
    response = slack_api_request(url, "GET", headers=headers, params=params)
    data = response.json()

    if response.status_code == 200 and data["ok"]:
        bot_id = "" 
        if bot_id not in data["members"]:
            invite_bot_to_channel(channel_id, auth)
    else:
        print("Error fetching channel members:", data.get("error", "Unknown error"))

        
# Invite the bot to the channel
def invite_bot_to_channel(channel_id, auth):
    url = "https://slack.com/api/conversations.join"
    headers = {
         "Authorization": f"Bearer {auth}"
    }
    data = {
        "channel": channel_id,
        "users": ""  
    }

    response = slack_api_request(url, "POST", headers=headers, data=data)
    data = response.json()

    if response.status_code == 200 and data["ok"]:
        print(f"Bot invited to channel {channel_id}")
    else:
        print("Error inviting bot to channel:", data.get("error", "Unknown error"))

        
# Scrapes Channel Messages
def scrape_channel(channel_id, auth):
    url = f"https://slack.com/api/conversations.history"
   
    headers = {
        "Authorization": f"Bearer {auth}"
    }
    params = {
        "channel": channel_id,
    }

    response = slack_api_request(url, "GET", headers=headers, params=params)
    data = response.json()

    if response.status_code == 200 and data["ok"]:
        return data["messages"]
    else:
        print("Error fetching messages:", data.get("error", "Unknown error"))
        return []


    # Scrapes Thread Messages
def scrape_threads(channel_id, ts, auth):
    url = f"https://slack.com/api/conversations.replies"
    headers = {
        "Authorization": f"Bearer {auth}"
    }
    params = {
        "channel": channel_id,
        "ts": ts,
    }

    response = slack_api_request(url, "GET", headers=headers, params=params)
    data = response.json()

    if response.status_code == 200 and data["ok"]:
        return data["messages"]
    else:
        print("Error fetching thread replies:", data.get("error", "Unknown error"))
        return []

def scrape(channel_id, auth):
    messages = scrape_channel(channel_id, auth)
    # Initialize a string to store messages and replies for the channel
    channel_text = ""

    # Iterate through messages
    for message in messages:
        question = message["text"]
        # Check if the message is a thread parent
        if "thread_ts" in message:
            thread_ts = message["thread_ts"]
            thread_replies = scrape_threads(channel_id, thread_ts, auth)
            for reply in thread_replies:
                # Append the question and reply to the channel_text string
                channel_text += f"Question: {question}\nAnswer: {reply['text']}\n\n"

    return channel_text

channels = channel_list(auth)
output_text = ""  # Initialize a string to store messages and replies for all channels

# Iterate through the Channel list and scrape each channel
for channel in channels:
    channel_id = channel["id"]
    ensure_bot_in_channel(channel_id, auth)
    print(f"\n\n**Scraping Messages from {channel['name']}**\n")
    channel_text = scrape(channel_id, auth)
    
    # Append the channel's text to the output_text string
    output_text += channel_text

# Save the output_text to a text file
output_txt_file_path = ""

with open(output_txt_file_path, "w", encoding="utf-8") as output_txt_file:
    output_txt_file.write(output_text)

print("done")
